In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/ML/Transfer Learning

/content/drive/MyDrive/ML/Transfer Learning


In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=92e5e21e2d19d6892b16342bc6ff4b1b5f3b824c59fba096465867a0d82f31cc
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget
link = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
wget.download(link)

In [ ]:
current_folder = "./"
!unzip -q cats_and_dogs_filtered.zip -d $current_folder

In [ ]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
  -O ./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-06-15 13:32:57--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

./inception_v3_weig 100%[===================>]  83.84M  43.4MB/s    in 1.9s    

2021-06-15 13:32:59 (43.4 MB/s) - ‘./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
local_weights_file = "./inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [ ]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape= (150, 150, 3),
                                include_top = False,
                                weights = None)
pre_trained_model.load_weights(local_weights_file)

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

Tiến hành đóng băng toàn bộ các layers

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

Xóa đi những layers nằm sau layer "mix7"

In [ ]:
last_layer = pre_trained_model.get_layer("mixed7")

In [ ]:
last_output = last_layer.output

In [ ]:
last_output

<KerasTensor: shape=(None, 7, 7, 768) dtype=float32 (created by layer 'mixed7')>

In [ ]:
from tensorflow.keras.optimizers import RMSprop

#Flatten the output layer into 1 dimension
x = layers.Flatten()(last_output)
#Add a fully connected layer with 1,024 hidden units using ReLU activation
x = layers.Dense(1024, activation= "relu")(x)
#Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
#Add a final sigmoid layer for classification
x = layers.Dense(1, activation= "sigmoid")(x)

model = Model(pre_trained_model.input, x)
model.compile(optimizer= RMSprop(learning_rate= 0.0001),
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

In [ ]:
train_folder = "./cats_and_dogs_filtered/train"
valid_folder = "./cats_and_dogs_filtered/validation"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#Note that the validation data should not be augmented
validation_datagen = ImageDataGenerator(rescale = 1.0/255.0)
#Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_folder,
                                                    batch_size = 20,
                                                    class_mode = "binary",
                                                    target_size = (150, 150))
#Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(valid_folder,
                                                              batch_size = 20,
                                                              class_mode = "binary",
                                                              target_size = (150, 150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'cats': 0, 'dogs': 1}

In [ ]:
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch = 100,
    #batch_size = 32
    epochs = 20,
    validation_steps = 50,
    verbose = 1
)

Epoch 1/20
100/100 [==============================] - 153s 2s/step - loss: 0.2143 - accuracy: 0.9145 - val_loss: 0.0935 - val_accuracy: 0.9690
Epoch 2/20
100/100 [==============================] - 154s 2s/step - loss: 0.1911 - accuracy: 0.9235 - val_loss: 0.1345 - val_accuracy: 0.9550
Epoch 3/20
100/100 [==============================] - 157s 2s/step - loss: 0.1864 - accuracy: 0.9365 - val_loss: 0.1456 - val_accuracy: 0.9510
Epoch 4/20
100/100 [==============================] - 154s 2s/step - loss: 0.2139 - accuracy: 0.9310 - val_loss: 0.1496 - val_accuracy: 0.9520
Epoch 5/20
100/100 [==============================] - 154s 2s/step - loss: 0.1778 - accuracy: 0.9350 - val_loss: 0.1219 - val_accuracy: 0.9620
Epoch 6/20
100/100 [==============================] - 154s 2s/step - loss: 0.1882 - accuracy: 0.9305 - val_loss: 0.1344 - val_accuracy: 0.9550
Epoch 7/20
100/100 [==============================] - 154s 2s/step - loss: 0.1679 - accuracy: 0.9405 - val_loss: 0.1182 - val_accuracy: 0.9660